In [1]:
import pandas as pd
import os
import folium
from folium.plugins import HeatMap

In [ ]:
download_maps = Fas

In [2]:
os.listdir('data')

['escolas_particulares.csv',
 'escolas_estaduais.csv',
 'praças.csv',
 'crimes.csv',
 'praças.xlsx',
 'restaurantes.csv',
 'escolas_municipais.csv',
 'outras_escolas.csv',
 'drive-download-20191029T202745Z-001.zip',
 'pontos_onibus.csv',
 'bares.csv']

In [3]:
crimes = pd.read_csv('data/crimes.csv', index_col=None)
escolas_particulares = pd.read_csv('data/escolas_particulares.csv', index_col=None)
escolas_estaduais = pd.read_csv('data/escolas_estaduais.csv', index_col=None)
pracas = pd.read_excel('data/praças.xlsx', index_col=None)
restaurantes = pd.read_csv('data/restaurantes.csv', index_col=None)
escolas_municipais = pd.read_csv('data/escolas_municipais.csv', index_col=None)
outras_escolas = pd.read_csv('data/outras_escolas.csv', index_col=None)
bares = pd.read_csv('data/bares.csv', index_col=None)
pontos_onibus = pd.read_csv('data/pontos_onibus.csv', index_col=None)

In [4]:
def generateBaseMap(default_location=[-22.010147, -47.890706], default_zoom_start=13):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

## Plotting Praças

In [5]:
map_pracas = generateBaseMap()
latitudes = list(pracas['LATITUDE'])
longitudes = list(pracas['LONGITUDE'])

cnt = 0
for lat, lng in zip(latitudes, longitudes):
    cnt += 1
    try:
        folium.Marker(location=[lat, lng], tooltip=[lat, lng]).add_to(map_pracas)
    except:
        print(f'{cnt} has no coordinates')

1 has no coordinates
3 has no coordinates
20 has no coordinates
22 has no coordinates
28 has no coordinates
38 has no coordinates
40 has no coordinates
43 has no coordinates
52 has no coordinates
54 has no coordinates
63 has no coordinates
66 has no coordinates
68 has no coordinates
69 has no coordinates
92 has no coordinates
95 has no coordinates
96 has no coordinates
97 has no coordinates
99 has no coordinates
104 has no coordinates
105 has no coordinates
106 has no coordinates


In [6]:
map_pracas

In [7]:
pracas_coordinates = pd.DataFrame(data=pracas)
try:
    pracas_coordinates = pracas_coordinates.drop(columns=['Nome', 'Descrição'])
except:
    print('Dropped already!')
    
pracas_coordinates = pracas_coordinates[pd.notnull(pracas_coordinates['LATITUDE']) & pd.notnull(pracas_coordinates['LONGITUDE'])]
pracas_heat_map = generateBaseMap()
HeatMap(data=pracas_coordinates[['LATITUDE', 'LONGITUDE']], radius=10, max_zoom=13).add_to(pracas_heat_map)

Dropped already!


In [8]:
pracas_heat_map

## Plotting Pontos de onibus

In [9]:
map_pontos = folium.Map(location=[-22.010147, -47.890706], zoom_start=13)
latitudes = list(pontos_onibus['Lat'])
longitudes = list(pontos_onibus['Long'])

cnt = 0
for lat, lng in zip(latitudes, longitudes):
    cnt += 1
    try:
        folium.Marker(location=[lat, lng], tooltip=[lat, lng]).add_to(map_pontos)
    except:
        print(f'{cnt} has no coordinates')

In [10]:
map_pontos

In [11]:
pontos_heat_map = generateBaseMap()
HeatMap(data=pontos_onibus[['Lat', 'Long']], radius=10, max_zoom=13).add_to(pontos_heat_map)

In [12]:
pontos_heat_map

## Plotting Bares

In [13]:
map_bares = folium.Map(location=[-22.010147, -47.890706], zoom_start=13)
latitudes = list(bares['LATITUDE'])
longitudes = list(bares['LONGITUDE'])

cnt = 0
for lat, lng in zip(latitudes, longitudes):
    cnt += 1
    try:
        folium.Marker(location=[lat, lng], tooltip=[lat, lng]).add_to(map_bares)
    except:
        print(f'{cnt} has no coordinates')

In [14]:
map_bares

In [15]:
bares_heat_map = generateBaseMap()
HeatMap(data=bares[['LATITUDE', 'LONGITUDE']], radius=10, max_zoom=13).add_to(bares_heat_map)

In [16]:
bares_heat_map

## Plotting Resturantes

In [17]:
map_restaurantes = folium.Map(location=[-22.010147, -47.890706], zoom_start=13)
latitudes = list(restaurantes['LATITUDE'])
longitudes = list(restaurantes['LONGITUDE'])

cnt = 0
for lat, lng in zip(latitudes, longitudes):
    cnt += 1
    try:
        folium.Marker(location=[lat, lng], tooltip=[lat, lng]).add_to(map_restaurantes)
    except:
        print(f'{cnt} has no coordinates')

In [18]:
map_restaurantes

In [19]:
restaurantes_heat_map = generateBaseMap()
HeatMap(data=restaurantes[['LATITUDE', 'LONGITUDE']], radius=10, max_zoom=13).add_to(restaurantes_heat_map)

In [20]:
restaurantes_heat_map

## Plotting everything

In [21]:
everything = pd.DataFrame(data=pontos_onibus)
everything = everything.rename(columns={'Lat' : 'LATITUDE', 'Long' : 'LONGITUDE'})
everything = pd.concat([everything, pracas_coordinates, bares, restaurantes], sort=False)
everything = everything[['LATITUDE', 'LONGITUDE']]
everything

,LATITUDE,LONGITUDE
0,-21.936774,-47.895920
1,-21.936067,-47.895156
2,-21.934742,-47.893912
3,-21.934391,-47.893440
4,-21.932259,-47.891519
...,...,...
259,-22.030438,-47.878881
260,-22.015727,-47.891877
261,-22.010371,-47.892051
262,-22.035546,-47.891158


In [22]:
all_heat_map = generateBaseMap()
HeatMap(data=everything,radius=10, max_zoom=13).add_to(all_heat_map)

In [23]:
all_heat_map

In [26]:
if download_maps:
    map_pracas.save(outfile='Pracas_points.html')
    pracas_heat_map.save(outfile='Pracas_heatMap.html')

    map_pontos.save(outfile='PontosOnibus_points.html')
    pontos_heat_map.save(outfile='PontosOnibus_heatMap.html')

    map_bares.save(outfile='Bares_points.html')
    bares_heat_map.save(outfile='Bares_heatMap.html')

    map_restaurantes.save(outfile='Restaurantes_points.html')
    restaurantes_heat_map.save(outfile='Restaurantes_heatMap.html')

    all_heat_map.save(outfile='Everything.html')